In [1]:
# Import all libraries
import folium
import numpy as np
import pandas as pd
from shapely.geometry import Point, LineString
import geopandas as gpd
from geopandas import GeoDataFrame


In [2]:
#insert data points & create delaunay triangulation using those points

points = np.array([[-6.305286, 106.644085], [-6.301309, 106.653261], [-6.284774, 106.637751], [-6.284598, 106.665062]])
from scipy.spatial import Delaunay
tri = Delaunay(points)

In [3]:
#create two empty dataframes, one to save every edges of the triangulation and the other one to save circumcenter for triangle

df = pd.DataFrame()
circumcenter = pd.DataFrame()

In [4]:
for x in range(points[tri.simplices].shape[0]):
    df = df.append(pd.DataFrame({'group': x+1,"geometry": [LineString([Point(points[tri.simplices][x][0][1], points[tri.simplices][x][0][0]), Point(points[tri.simplices][x][1][1], points[tri.simplices][x][1][0])])]}))
    df = df.append(pd.DataFrame({'group': x+1,"geometry": [LineString([Point(points[tri.simplices][x][1][1], points[tri.simplices][x][1][0]), Point(points[tri.simplices][x][2][1], points[tri.simplices][x][2][0])])]}))
    df = df.append(pd.DataFrame({'group': x+1,"geometry": [LineString([Point(points[tri.simplices][x][2][1], points[tri.simplices][x][2][0]), Point(points[tri.simplices][x][0][1], points[tri.simplices][x][0][0])])]}))
    # find lat & lon coordinate for three vertices of the triangle
    circumcenter = circumcenter.append(pd.DataFrame({'group': x+1,
                                                     'point_1_lon': points[tri.simplices][x][0][1],
                                                     'point_1_lat': points[tri.simplices][x][0][0],
                                                     'point_2_lon': points[tri.simplices][x][1][1],
                                                     'point_2_lat': points[tri.simplices][x][1][0],
                                                     'point_3_lon': points[tri.simplices][x][2][1],
                                                     'point_3_lat': points[tri.simplices][x][2][0]
                                                    }, index=[0]))


In [5]:
circumcenter

,group,point_1_lon,point_1_lat,point_2_lon,point_2_lat,point_3_lon,point_3_lat
0,1,106.637751,-6.284774,106.653261,-6.301309,106.644085,-6.305286
0,2,106.653261,-6.301309,106.637751,-6.284774,106.665062,-6.284598


In [6]:
#find mid point for each edge
df.reset_index(drop=True)
df['mid_point'] = df.apply(lambda row: row['geometry'].interpolate(0.5, normalized = True),axis=1)

In [7]:
gdf = GeoDataFrame(df,geometry='geometry')

In [8]:
#find x and y posistion for start and enpoint of each edge
def find_x1(row):
    x,y = row.coords.xy
    x1,x2 = x
    return x1
def find_x2(row):
    x,y = row.coords.xy
    x1,x2 = x
    return x2
def find_y1(row):
    x,y = row.coords.xy
    y1,y2 = y
    return y1
def find_y2(row):
    x,y = row.coords.xy
    y1,y2 = y
    return y2

gdf['x1'] = gdf.apply(lambda row:find_x1(row['geometry']),axis=1)
gdf['x2'] = gdf.apply(lambda row:find_x2(row['geometry']),axis=1)
gdf['y1'] = gdf.apply(lambda row:find_y1(row['geometry']),axis=1)
gdf['y2'] = gdf.apply(lambda row:find_y2(row['geometry']),axis=1)

In [9]:
def circumcenter_triangle(ax,ay,bx,by,cx,cy):
    d = 2 * (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by))
    ux = ((ax * ax + ay * ay) * (by - cy) + (bx * bx + by * by) * (cy - ay) + (cx * cx + cy * cy) * (ay - by)) / d
    uy = ((ax * ax + ay * ay) * (cx - bx) + (bx * bx + by * by) * (ax - cx) + (cx * cx + cy * cy) * (bx - ax)) / d
    return Point(ux, uy)

In [10]:
# find circumcenter for each triangle
circumcenter['circumcenter_triangle'] = circumcenter.apply(lambda row: circumcenter_triangle(row['point_1_lon'],row['point_1_lat'],row['point_2_lon'],row['point_2_lat'],row['point_3_lon'],row['point_3_lat']) ,axis=1)

In [11]:
# find circumcenter for each circumcenter
circumcenter['circumcenter_triangle_lon'] = circumcenter.apply(lambda row: row['circumcenter_triangle'].x,axis=1)
circumcenter['circumcenter_triangle_lat'] = circumcenter.apply(lambda row: row['circumcenter_triangle'].y,axis=1)

In [12]:
# get lon lat for each mid points
gdf['mid_point_lon'] = gdf.apply(lambda row: row['mid_point'].x,axis=1)
gdf['mid_point_lat'] = gdf.apply(lambda row: row['mid_point'].y,axis=1)

In [13]:
circumcenter

,group,point_1_lon,point_1_lat,point_2_lon,point_2_lat,point_3_lon,point_3_lat,circumcenter_triangle,circumcenter_triangle_lon,circumcenter_triangle_lat
0,1,106.637751,-6.284774,106.653261,-6.301309,106.644085,-6.305286,POINT (106.6445973299853 -6.293893841737034),106.644597,-6.293894
0,2,106.653261,-6.301309,106.637751,-6.284774,106.665062,-6.284598,POINT (106.6514245686324 -6.287489821777483),106.651425,-6.287490


In [14]:
#create new dataframe to save perpendicular lines

perpendicular_line = pd.merge(gdf[['group','mid_point','mid_point_lon','mid_point_lat']], circumcenter[['group','circumcenter_triangle','circumcenter_triangle_lon','circumcenter_triangle_lat']], on='group')
perpendicular_line['geometry'] = perpendicular_line.apply(lambda row:LineString([row['mid_point'], row['circumcenter_triangle']]),axis=1)
perpendicular_line = GeoDataFrame(perpendicular_line,geometry='geometry')

In [15]:
perpendicular_line

,group,mid_point,mid_point_lon,mid_point_lat,circumcenter_triangle,circumcenter_triangle_lon,circumcenter_triangle_lat,geometry
0,1,POINT (106.645506 -6.293041499999999),106.645506,-6.293041,POINT (106.6445973299853 -6.293893841737034),106.644597,-6.293894,"LINESTRING (106.64551 -6.29304, 106.64460 -6.2..."
1,1,POINT (106.648673 -6.303297499999999),106.648673,-6.303297,POINT (106.6445973299853 -6.293893841737034),106.644597,-6.293894,"LINESTRING (106.64867 -6.30330, 106.64460 -6.2..."
2,1,POINT (106.640918 -6.29503),106.640918,-6.295030,POINT (106.6445973299853 -6.293893841737034),106.644597,-6.293894,"LINESTRING (106.64092 -6.29503, 106.64460 -6.2..."
3,2,POINT (106.645506 -6.293041499999999),106.645506,-6.293041,POINT (106.6514245686324 -6.287489821777483),106.651425,-6.287490,"LINESTRING (106.64551 -6.29304, 106.65142 -6.2..."
4,2,POINT (106.6514065 -6.284686),106.651407,-6.284686,POINT (106.6514245686324 -6.287489821777483),106.651425,-6.287490,"LINESTRING (106.65141 -6.28469, 106.65142 -6.2..."
5,2,POINT (106.6591615 -6.292953499999999),106.659162,-6.292953,POINT (106.6514245686324 -6.287489821777483),106.651425,-6.287490,"LINESTRING (106.65916 -6.29295, 106.65142 -6.2..."


In [16]:
#find slope & y-intercept
perpendicular_line['slope'] = perpendicular_line.apply(lambda row: (row['mid_point_lat']-row['circumcenter_triangle_lat'])/(row['mid_point_lon']-row['circumcenter_triangle_lon']),axis=1)
perpendicular_line['y_intercept'] = perpendicular_line.apply(lambda row: row['mid_point_lat'] - (row['slope'] * row['mid_point_lon']),axis=1)


In [17]:
perpendicular_line

,group,mid_point,mid_point_lon,mid_point_lat,circumcenter_triangle,circumcenter_triangle_lon,circumcenter_triangle_lat,geometry,slope,y_intercept
0,1,POINT (106.645506 -6.293041499999999),106.645506,-6.293041,POINT (106.6445973299853 -6.293893841737034),106.644597,-6.293894,"LINESTRING (106.64551 -6.29304, 106.64460 -6.2...",0.938010,-106.327613
1,1,POINT (106.648673 -6.303297499999999),106.648673,-6.303297,POINT (106.6445973299853 -6.293893841737034),106.644597,-6.293894,"LINESTRING (106.64867 -6.30330, 106.64460 -6.2...",-2.307267,239.763649
2,1,POINT (106.640918 -6.29503),106.640918,-6.295030,POINT (106.6445973299853 -6.293893841737034),106.644597,-6.293894,"LINESTRING (106.64092 -6.29503, 106.64460 -6.2...",0.308795,-39.225200
3,2,POINT (106.645506 -6.293041499999999),106.645506,-6.293041,POINT (106.6514245686324 -6.287489821777483),106.651425,-6.287490,"LINESTRING (106.64551 -6.29304, 106.65142 -6.2...",0.938010,-106.327622
4,2,POINT (106.6514065 -6.284686),106.651407,-6.284686,POINT (106.6514245686324 -6.287489821777483),106.651425,-6.287490,"LINESTRING (106.65141 -6.28469, 106.65142 -6.2...",-155.176204,16543.475720
5,2,POINT (106.6591615 -6.292953499999999),106.659162,-6.292953,POINT (106.6514245686324 -6.287489821777483),106.651425,-6.287490,"LINESTRING (106.65916 -6.29295, 106.65142 -6.2...",-0.706182,69.027779


In [18]:
# determine the coverage area of the voronoi diagram

area_max_lon = 106.670929
area_min_lon = 106.619602
area_max_lat = -6.275713
area_min_lat = -6.309795

coords = [[area_min_lon, area_min_lat], [ area_min_lon, area_max_lat], [ area_max_lon,area_max_lat], [ area_max_lon,area_min_lat]]


In [19]:
# extend the perpendicular lines so it can fit the size of area

perpendicular_line.loc[perpendicular_line['circumcenter_triangle_lat']>perpendicular_line['mid_point_lat'], 'ext_lat'] = area_min_lat
perpendicular_line.loc[perpendicular_line['circumcenter_triangle_lat']<perpendicular_line['mid_point_lat'], 'ext_lat'] = area_max_lat

perpendicular_line['ext_lon'] = perpendicular_line.apply(lambda row: (row['ext_lat']-row['y_intercept'])/row['slope'],axis=1)

perpendicular_line.loc[(perpendicular_line['ext_lon']>area_max_lon) , 'ext_lon'] = area_max_lon
perpendicular_line.loc[(perpendicular_line['ext_lon']<area_min_lon) , 'ext_lon'] = area_min_lon

perpendicular_line['ext_lat'] = perpendicular_line.apply(lambda row: (row['ext_lon']*row['slope'])+row['y_intercept'],axis=1)


perpendicular_line.loc[perpendicular_line.duplicated(['mid_point_lon', 'mid_point_lat'],keep=False), 'ext_lon'] = perpendicular_line['mid_point_lon']
perpendicular_line.loc[perpendicular_line.duplicated(['mid_point_lon', 'mid_point_lat'],keep=False), 'ext_lat'] = perpendicular_line['mid_point_lat']



In [20]:
perpendicular_line['ext_point'] = perpendicular_line.apply(lambda row:Point(row['ext_lon'],row['ext_lat']),axis=1)


perpendicular_line['geometry'] = perpendicular_line.apply(lambda row:LineString([row['ext_point'], row['circumcenter_triangle']]),axis=1)
perpendicular_line = GeoDataFrame(perpendicular_line,geometry='geometry')


In [21]:
perpendicular_line

,group,mid_point,mid_point_lon,mid_point_lat,circumcenter_triangle,circumcenter_triangle_lon,circumcenter_triangle_lat,geometry,slope,y_intercept,ext_lat,ext_lon,ext_point
0,1,POINT (106.645506 -6.293041499999999),106.645506,-6.293041,POINT (106.6445973299853 -6.293893841737034),106.644597,-6.293894,"LINESTRING (106.64551 -6.29304, 106.64460 -6.2...",0.938010,-106.327613,-6.293041,106.645506,POINT (106.645506 -6.293041499999999)
1,1,POINT (106.648673 -6.303297499999999),106.648673,-6.303297,POINT (106.6445973299853 -6.293893841737034),106.644597,-6.293894,"LINESTRING (106.65149 -6.30980, 106.64460 -6.2...",-2.307267,239.763649,-6.309795,106.651489,POINT (106.6514891025401 -6.309795000000008)
2,1,POINT (106.640918 -6.29503),106.640918,-6.295030,POINT (106.6445973299853 -6.293893841737034),106.644597,-6.293894,"LINESTRING (106.61960 -6.30161, 106.64460 -6.2...",0.308795,-39.225200,-6.301612,106.619602,POINT (106.619602 -6.301612271671701)
3,2,POINT (106.645506 -6.293041499999999),106.645506,-6.293041,POINT (106.6514245686324 -6.287489821777483),106.651425,-6.287490,"LINESTRING (106.64551 -6.29304, 106.65142 -6.2...",0.938010,-106.327622,-6.293041,106.645506,POINT (106.645506 -6.293041499999999)
4,2,POINT (106.6514065 -6.284686),106.651407,-6.284686,POINT (106.6514245686324 -6.287489821777483),106.651425,-6.287490,"LINESTRING (106.65135 -6.27571, 106.65142 -6.2...",-155.176204,16543.475720,-6.275713,106.651349,POINT (106.6513486754124 -6.275712999999087)
5,2,POINT (106.6591615 -6.292953499999999),106.659162,-6.292953,POINT (106.6514245686324 -6.287489821777483),106.651425,-6.287490,"LINESTRING (106.67093 -6.30126, 106.65142 -6.2...",-0.706182,69.027779,-6.301263,106.670929,POINT (106.670929 -6.301263491445965)


In [22]:

gdf.crs = {'init' :'epsg:4326'}
perpendicular_line.crs = {'init' :'epsg:4326'}
m = folium.Map((-6.304029, 106.645874),
zoom_start=13,
tiles="CartoDb dark_matter")

#draw the area
line = {'type': 'Feature',
        'geometry': {'type': 'LineString',
                     'coordinates': coords }
        }

folium.GeoJson(line, name='line',
               style_function=lambda x: {'color': '#999999',
                                         'fill': '#999999',
                                         'opacity': 0.8,
                                         'fillOpacity': 0.5}).add_to(m)

#draw the voronoi diagram
folium.GeoJson(perpendicular_line['geometry'], style_function=lambda x: {'color': '#e6e31d', 'weight':'1'}).add_to(m)


#draw the data points
locs = points
for location in locs:
    folium.CircleMarker(location=location, 
        color = "#F4F6F7",   radius=0.01).add_to(m)

    

m

/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
